### 1. Imports

In [1]:
import requests
import json
import pandas as pd

### 2. Get user input 

In [2]:
# Create dict of available teams
team_dict = {'bayern': 40,
             'dortmund': 7,
             'schalke': 9}

In [3]:
home_team = input("What's the name of the home team?")
home_team = home_team.lower()

What's the name of the home team? bayern


In [4]:
away_team = input("What's the name of the away team?")
away_team = away_team.lower()

What's the name of the away team? dortmund


In [5]:
# Check if team exists
def check_team(home_team, away_team, team_dict):
    if home_team in team_dict and away_team in team_dict:
        home = team_dict[home_team]
        away = team_dict[away_team]
        return home, away
    else:
        return None, None

In [6]:
home, away = check_team(home_team, away_team, team_dict)

### 3. API call / get the data

In [7]:
# Get the match data for both teams: team1 is the HOME and team2 is the AWAY team
url = f'https://api.openligadb.de/getmatchdata/{home}/{away}'
response = requests.get(url)

In [129]:
endergebnis = 'Endergebnis'

In [202]:
# check if result at the end of the match
endergebnis = 'Endergebnis'
# count number of matches
match = 0
# store if win, draw, loss in dict
match_history = {'match':[],
                 'outcome': [],
                 'goals_team_1': [],
                 'goals_team_2': []}
# store match_history dict in list of dicts to later transform into df
match_list = []

for x in range(0, len(response.json())):
    
    try:
        if response.json()[x]['team1']['teamId'] == home:
            if endergebnis in response.json()[x]['matchResults'][0].values():
                goals_team1 = response.json()[x]['matchResults'][0]['pointsTeam1']
                goals_team2 = response.json()[x]['matchResults'][0]['pointsTeam2']
                if goals_team1 > goals_team2:
                    match_history['match'].append(x)
                    match_history['outcome'].append(1)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    #match_history[f'match_{match}': {}] = 1,
                    match += 1
                elif goals_team1 < goals_team2:
                    match_history['match'].append(x)
                    match_history['outcome'].append(2)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    #match_history[f'match_{match}'] = 2
                    match += 1
                else:
                    match_history['match'].append(x)
                    match_history['outcome'].append(0)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    #match_history[f'match_{match}'] = 0
                    match += 1
            else:
                goals_team1 = response.json()[x]['matchResults'][1]['pointsTeam1']
                goals_team2 = response.json()[x]['matchResults'][1]['pointsTeam2']
                if goals_team1 > goals_team2:
                    match_history['match'].append(x)
                    match_history['outcome'].append(1)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    match += 1
                elif goals_team1 < goals_team2:
                    match_history['match'].append(x)
                    match_history['outcome'].append(2)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    match += 1
                else: 
                    match_history['match'].append(x)
                    match_history['outcome'].append(0)
                    match_history['goals_team_1'].append(goals_team1)
                    match_history['goals_team_2'].append(goals_team2)
                    match += 1
                     
        #print(goals_team1)
            #scores_df['score_team_1'] = goals_team1
            #scores_df['score_team_2'] = goals_team2
    except IndexError:
        print(response.json()[x]['matchID'])
        print(f'Number: {x}')
        print('nothingburger')
        continue

4682
Number: 0
nothingburger
64090
Number: 72
nothingburger
64278
Number: 73
nothingburger


In [179]:
match_history

{'match': [3,
  5,
  6,
  9,
  12,
  14,
  15,
  18,
  22,
  24,
  25,
  28,
  29,
  31,
  34,
  35,
  41,
  42,
  43,
  47,
  49,
  56,
  62,
  67,
  69],
 'outcome': [1,
  0,
  1,
  1,
  1,
  2,
  2,
  0,
  2,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  0,
  1],
 'goals_team_1': [5,
  3,
  2,
  5,
  3,
  1,
  0,
  1,
  0,
  2,
  2,
  1,
  1,
  5,
  4,
  4,
  4,
  2,
  2,
  2,
  6,
  5,
  4,
  0,
  3],
 'goals_team_2': [0,
  3,
  0,
  0,
  1,
  3,
  1,
  1,
  3,
  1,
  1,
  3,
  3,
  1,
  3,
  3,
  1,
  3,
  3,
  1,
  0,
  0,
  2,
  0,
  1],
 'match_13': 1,
 'match_17': 1,
 'match_21': 1,
 'match_23': 1,
 'match_25': 1,
 'match_27': 1}

In [217]:
match_df = pd.DataFrame(match_history.items())#, columns=['match_number', 'outcome', 'goals_team_1', 'goals_team_2'])

In [218]:
df = match_df.set_index(0).T

In [221]:
df

,match,outcome,goals_team_1,goals_team_2
1,"[3, 5, 6, 9, 12, 14, 15, 18, 22, 24, 25, 28, 2...","[1, 0, 1, 1, 1, 2, 2, 0, 2, 1, 1, 2, 2, 1, 1, ...","[5, 3, 2, 5, 3, 1, 0, 1, 0, 2, 2, 1, 1, 5, 5, ...","[0, 3, 0, 0, 1, 3, 1, 1, 3, 1, 1, 3, 3, 1, 1, ..."


In [223]:
df = df.explode(['match', 'outcome', 'goals_team_1', 'goals_team_2'])

In [227]:
df = df[['match', 'goals_team_1', 'goals_team_2', 'outcome']]

In [235]:
df = df.set_index('match').reset_index()

In [236]:
df

,match,goals_team_1,goals_team_2,outcome
0,3,5,0,1
1,5,3,3,0
2,6,2,0,1
3,9,5,0,1
4,12,3,1,1
5,14,1,3,2
6,15,0,1,2
7,18,1,1,0
8,22,0,3,2
9,24,2,1,1
